# Stromingen example: keeping scripts simple

---

This example is based on the essay _"Open source grondwatermodellering met
MODFLOW 6"_ that was recently published in Stromingen (Calje et al., 2022).

This example strives to achieve the simplicity of the example psuedo script
that was shown in Figure 5 in the article. Some things require a bit more code
than in the example, but not by much! Also some data is not yet publicly
accessible, i.e. well data, so that has also not yet been implemented in this
example. We also changed the extent to build a smaller model (because of
computation time).

### Import Python packages

In [ ]:
import os
import flopy as fp
import geopandas as gpd
import nlmod
from pandas import date_range

nlmod.util.get_color_logger("INFO")
print(f"nlmod version: {nlmod.__version__}")

### Define spatial and temporal properties

In [ ]:
extent = [116_500, 120_000, 439_000, 442_000]
tmin = "2010-01-01"
tmax = "2020-01-01"
freq = "14D"

In [ ]:
# where in the world are we?
nlmod.plot.get_map(extent, background=True);

### Get data for the current extent

In [ ]:
# define a model workspace for caching downloaded data
model_ws = "schoonhoven"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)

In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent,
    use_regis=True,
    use_geotop=False,
    cachedir=cachedir,
    cachename="layer_model.nc",
)

# wells = nlmod.read.get_wells(extent)  # no well data is available just yet

# surface water features and levels
fname_bgt = os.path.join(cachedir, "bgt.geojson")
if not os.path.isfile(fname_bgt):
    raise (
        Exception(
            f"{fname_bgt} not found. Please run notebook 02_surface_water.ipynb first"
        )
    )
sw = gpd.read_file(fname_bgt)

### Generate grid with a refinement zone

In [ ]:
# create a model dataset
ds = nlmod.to_model_ds(layer_model, "stromingen", model_ws=model_ws)

# refine model dataset (supply a list of xy-coordinates)
xy = [
    [
        [
            (117_500, 439_500),
            (117_500, 440_000),
            (118_000, 440_000),
            (118_000, 439_500),
            (117_500, 439_500),
        ]
    ]
]

refinement = [(xy, "polygon", 1)]
ds = nlmod.grid.refine(ds, refinement_features=refinement)

### Generate a model

In [ ]:
# set model time settings
t = date_range(tmin, tmax, freq=freq)
ds = nlmod.time.set_ds_time(ds, start=3652, time=t, steady_start=True)

# build the modflow6 gwf model
gwf = nlmod.gwf.ds_to_gwf(ds)

### Add recharge to the model

In [ ]:
# download knmi recharge data
knmi_ds = nlmod.read.knmi.get_recharge(ds)

# update model dataset
ds.update(knmi_ds)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

### Add surface water to the model


In [ ]:
# set stage for model to mean of summer and winter levels
sw["stage"] = sw[["winter_stage", "summer_stage"]].mean(axis=1)

# use a water depth of 0.5 meter
sw["rbot"] = sw["stage"] - 0.5

# set the stage of the Lek river to 0.0 m NAP and the botm to -3 m NAP
mask = sw["bronhouder"] == "L0002"
sw.loc[mask, "stage"] = 0.0
sw.loc[mask, "rbot"] = -3.0

# we need to mask out the NaNs
sw.drop(sw.index[sw["rbot"].isna()], inplace=True)

In [ ]:
# intersect surface water with grid
sfw_grid = nlmod.grid.gdf_to_grid(sw, gwf)

# add bed resistance to calculate conductance
bed_resistance = 1.0  # days
sfw_grid["cond"] = sfw_grid.area / bed_resistance
sfw_grid.set_index("cellid", inplace=True)

# build stress period data for RIV package
riv_spd = nlmod.gwf.surface_water.build_spd(sfw_grid, "RIV", ds)

In [ ]:
# flopy is used to construct the RIV package directly
riv = fp.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd)

### Run model

In [ ]:
nlmod.sim.write_and_run(gwf, ds, silent=True)

### Post-processing

Plotting the average head in REGIS layer PZWAz3

In [ ]:
# load the computed heads
head = nlmod.gwf.output.get_heads_da(ds)

In [ ]:
# plot on map
ax = nlmod.plot.map_array(
    head.sel(layer="PZWAz3").mean(dim="time"),
    ds,
    alpha=0.25,
    background=True,
)

Plot the GHG in the upper layer, named 'HLc'.

In [ ]:
# use nlmod's calculate_gxg method to calculate the GVG, GLG and GHG.
gxg = nlmod.gwf.calculate_gxg(head.sel(layer="HLc"))

# plot on map
pc = nlmod.plot.map_array(gxg["ghg"], ds)

### References

- Calje, R., F. Schaars, D. Brakenhoff, O. Ebbens. (2022) "Open source grondwatermodellering met MODFLOW 6". Stromingen 2022 Vol. 03.